In [4]:
# import necessary packages of pytorch required to make the model for object detection and classification
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.utils as vutils
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [7]:
# create class to load the data and preprocess the data
class LoadData:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.train_dir = data_dir + '/train'
        self.valid_dir = data_dir + '/valid'
        self.test_dir = data_dir + '/test'
        self.train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                                         [0.229, 0.224, 0.225])])
        self.valid_transforms = transforms.Compose([transforms.Resize(256),
                                                    transforms.CenterCrop(224),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                                         [0.229, 0.224, 0.225])])
        self.test_transforms = transforms.Compose([transforms.Resize(256),
                                                   transforms.CenterCrop(224),
                                                   transforms.ToTensor(),
                                                   transforms.Normalize([0.485, 0.456, 0.406],
                                                                        [0.229, 0.224, 0.225])])
        self.train_data = datasets.ImageFolder(self.train_dir, transform=self.train_transforms)
        self.valid_data = datasets.ImageFolder(self.valid_dir, transform=self.valid_transforms)
        self.test_data = datasets.ImageFolder(self.test_dir, transform=self.test_transforms)
        self.train_loader = torch.utils.data.DataLoader(self.train_data, batch_size=32, shuffle=True)
        self.valid_loader = torch.utils.data.DataLoader(self.valid_data, batch_size=32, shuffle=True)
        self.test_loader = torch.utils.data.DataLoader(self.test_data, batch_size=32, shuffle=True)
        self.classes = self.train_data.classes
        self.num_classes = len(self.classes)
        self.class_to_idx = self.train_data.class_to_idx
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}
        self.train_data_size = len(self.train_data)
        self.valid_data_size = len(self.valid_data)
        self.test_data_size = len(self.test_data)
    
    # function to load the data
    def load_data(self):
        return self.train_loader, self.valid_loader, self.test_loader, self.classes, self.num_classes, self.class_to_idx, self.idx_to_class, self.train_data_size, self.valid_data_size, self.test_data_size

    # function to load the data in batches
    def load_data_in_batches(self, batch_size):
        return torch.utils.data.DataLoader(self.train_data, batch_size=batch_size, shuffle=True)

    # function to train the data
    def train_data(self):
        return self.train_loader

    # function to validate the data
    def valid_data(self):
        return self.valid_loader

    # function to test the data
    def test_data(self):
        return self.test_loader
 

In [10]:
# load data 
data_dir = 'dataset/train/images/12114_jpg.rf.6b8206b39679d8e1c8e6a7bcb91073c7.jpg'
img = cv2.imread(data_dir)
plt.imshow(img)

# data = LoadData(data_dir)
# train_loader, valid_loader, test_loader, classes, num_classes, class_to_idx, idx_to_class, train_data_size, valid_data_size, test_data_size = data.load_data()


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/train/train'